In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
DMRs = pd.read_table("./DMR_annotation_beta.40.3_ratio.1.bed", header=None)
DMRs.index = DMRs[0]+'.'+DMRs[1].astype(str)+'.'+DMRs[2].astype(str)
DMRs

In [ ]:
pd.crosstab(DMRs[6],DMRs[7])

In [ ]:
inDMRs = {}
inDMRs2 = {}
for i in DMRs.sample(200, random_state=1).index:
    for j in range(int(DMRs[1][i]), int(DMRs[2][i]+1)):
        inDMRs[j] = DMRs[6][i]
        inDMRs2[j] = DMRs[3][i]

In [ ]:
DMR_order = {
    '0,1,2|3,4':0, 
    '0|others':1, 
    '1|others':2, 
    '2|others':3, 
    '3|others':4, 
    '4|others':5,
    '0|1,2':6, 
    '3|4':7,
}

In [ ]:
beta = pd.read_table('./beta.40.3_ratio.1.5grps.metilene')
beta['DMR'] = beta['pos'].map(inDMRs)
beta['DMRo'] = beta['DMR'].map(DMR_order)
beta['diff'] = beta['pos'].map(inDMRs2)
beta = beta.dropna().sort_values(['DMRo','diff'])
beta

In [ ]:
pd.crosstab(beta['DMR'],beta['DMRo'])

In [ ]:
sns.clustermap(beta[beta.columns[2:-3]],
               row_cluster=False,
               cmap='coolwarm',
               figsize=[2,12])

In [ ]:
sns.clustermap(beta[beta['DMR']=='0,1,2|3,4'][beta.columns[2:-2]],row_cluster=False,cmap='coolwarm')

In [ ]:
Predictions = pd.read_table("../../beta.40.3_ratio.1.5grps.metilene.res", encoding='iso-8859-1')
Predictions.index = Predictions['chr']+'.'+Predictions['start'].astype(str)+'.'+Predictions['stop'].astype(str)
df_res = pd.merge(DMRs, Predictions, left_index=True, right_index=True, how='outer').fillna('NA')
pd.crosstab(df_res[6], df_res['sig.comparison'])